# IMPORT LIBRARY

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb
from tqdm import tqdm
import pandas as pd
import random
import os
import numpy as np

from sklearn.model_selection import GridSearchCV 
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
import sklearn.metrics as metrics

import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


# DATA LOAD

In [49]:
# Load the dataset
file_path = '../train_data/merge_train_preprocess_refine_columns.csv'
df = pd.read_csv(file_path)


In [54]:
data = df.copy()

## sampling 전 val data 추출

In [58]:
y = df['SUBCLASS']

In [59]:
# Split the data
data, X_test, y_data, y_test = train_test_split(df, y, stratify=y, test_size=0.05, random_state=42)


In [60]:
data.head()

,SUBCLASS,ID,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
1818,COAD,TRAIN_1818,WT,E192E,V928fs,WT,R1482C A3261V,WT,WT,WT,...,WT,WT,I250V,WT,WT,WT,WT,WT,WT,WT
5410,KIPAN,TRAIN_5410,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
12275,THCA,TCGA-EM-A2OX-01A-11D-A202-08,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
373,KIPAN,TRAIN_0373,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3112,UCEC,TRAIN_3112,WT,WT,WT,WT,L1469Q,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,A403T,WT


# DATA SAMPLING

In [61]:
# 목표 데이터 수 설정
target_count = 600

# 각 subclass별로 200개의 데이터를 추출하고, 부족한 경우 복제해서 채움
balanced_data = data.groupby('SUBCLASS', group_keys=False).apply(
    lambda x: x.sample(n=target_count, replace=True, random_state=1)
)

# 결과 확인
print(balanced_data['SUBCLASS'].value_counts())

# 데이터 저장
# balanced_data.to_csv('../train_data/balanced_data_200.csv', index=False)


SUBCLASS
ACC       600
BLCA      600
THYM      600
THCA      600
TGCT      600
STES      600
SKCM      600
SARC      600
PRAD      600
PCPG      600
PAAD      600
OV        600
LUSC      600
LUAD      600
LIHC      600
LGG       600
LAML      600
KIRC      600
KIPAN     600
HNSC      600
GBMLGG    600
DLBC      600
COAD      600
CESC      600
BRCA      600
UCEC      600
Name: count, dtype: int64


C:\Users\JINWOO\AppData\Local\Temp\ipykernel_25532\814961623.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_data = data.groupby('SUBCLASS', group_keys=False).apply(


In [62]:

# Combine all features into a single string per sample for TF-IDF transformation
data_features = balanced_data.drop(columns=['SUBCLASS', 'ID'])

# 전처리 함수 정의

## 변이 된 값에 해당 컬럼 넣기

In [63]:
import re

def transform_mutation(mutation, col_name):
    # 변이 문자열의 패턴을 확인하고 해당하는 변이 유형을 반환
    if mutation == 'WT':
        return ''
    elif re.match(r"^[A-Z](\d+)[A-Z]$", mutation):
        return '' if mutation[0] == mutation[-1] else col_name
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)delins([A-Z]+)", mutation):
        return col_name
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)ins([A-Z]+)", mutation):
        return col_name
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)del", mutation):
        return f"{col_name} "*5
    elif re.match(r"([A-Z])(\d+)\*", mutation):
        return f"{col_name} " * 10
    elif re.match(r"([A-Z])(\d+)fs", mutation):
        return f"{col_name} " * 10
    elif re.match(r"(\d+)_\d+[A-Z]+>([A-Z]+)", mutation):
        original = mutation.split(">")[0]
        target = mutation.split(">")[1]
        if original == target:
            return ''
        return col_name
    else:
        return col_name  # 기본적으로 '해당 컬럼' 반환


## 위 함수를 이용한 전처리

In [64]:
# 데이터프레임의 모든 변이 관련 열에 전처리 함수 적용
for col in data_features:
    data_features[col] = data_features[col].apply(lambda x: transform_mutation(x, col))

# 변환된 데이터프레임 출력
data_features.head()


,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,AQP7,PIM1,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
2920,,,,,,,,,,,...,,,,,,PABPC1,,,,
6214,,,,,,,,,,,...,,,,,,,,,,
6242,,,,,,,,,,,...,,,,,,,,,,
6210,,,,,,,,,,,...,,,,,,,,,,
734,,,,,,,,,,,...,,,,,,,,,,


## 변이 된 값에 가중치 넣기

In [42]:
import re

def classify_mutation(mutation):
    # 변이 문자열의 패턴을 확인하고 해당하는 변이 유형을 반환
    if mutation == 'WT':
        return 0
    elif re.match(r"^[A-Z](\d+)[A-Z]$", mutation):
        return 0 if mutation[0] == mutation[-1] else 0.2
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)delins([A-Z]+)", mutation):
        return 0.2
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)ins([A-Z]+)", mutation):
        return 0.2
    
    elif re.match(r"([A-Z])(\d+)_([A-Z])(\d+)del", mutation):
        return 0.5
    elif re.match(r"([A-Z])(\d+)\*", mutation):
        return 1
    elif re.match(r"([A-Z])(\d+)fs", mutation):
        return 1
    elif re.match(r"(\d+)_\d+[A-Z]+>([A-Z]+)", mutation):
        original = mutation.split(">")[0]
        target = mutation.split(">")[1]
        if original == target:
            return 0
        return 0.2
    else:
        return 1


In [43]:
for col in data_features.columns:
    data_features[col] = data_features[col].apply(classify_mutation)
    
# 변환된 데이터프레임 출력
data_features.head()

,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,AQP7,PIM1,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
3355,0.0,0.0,0.0,0.2,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# TFvectorizer 이용하여 X값 encoding

## vectorizer에 넣기 전에 행을 하나의 string으로 만들기

In [65]:
data_vec = data_features.astype(str).apply(' '.join, axis=1)

## vectorizer 사용하여 encoding

In [66]:
# Convert combined feature strings to TF-IDF features
tfidf_vectorizer = TfidfVectorizer(max_features=1500)
X_tfidf = tfidf_vectorizer.fit_transform(data_vec)

# Label encoding

In [67]:
# Extract labels
y = balanced_data['SUBCLASS']

In [68]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

## x와 y 값 shape 비교

In [69]:
# X_tfidf와 y_encoded의 크기를 확인
print(f"X_tfidf shape: {X_tfidf.shape}")
print(f"y_encoded shape: {len(y_encoded)}")

X_tfidf shape: (15600, 148)
y_encoded shape: 15600


# 모델 학습

## 1. 데이터 분리

In [13]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_encoded, stratify=y_encoded, test_size=0.1, random_state=42)


## 2-1) xgboost 단일 모델 학습

In [71]:
from xgboost import XGBClassifier

In [72]:

model = xgb.XGBClassifier(
    n_estimators=800,
    learning_rate=0.05,
    max_depth=18,
    colsample_bytree = 0.4,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

AttributeError: 'XGBClassifier' object has no attribute 'XGBClassifier'

In [15]:
model.fit(X_train, y_train)

c:\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [02:06:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=600,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

## 2-2) xgboost GridSearch

In [26]:
from xgboost import XGBClassifier

#XGBoost 정의
#파라미터는 이전 글을 참고하여 grid 설정(dict 형태)
xgb= XGBClassifier()
xgb_param_grid={
    'n_estimators' : [400, 500, 600, 700, 800],     
    'learning_rate' : [0.1,0.05],    
    'max_depth' : [14,15,16,17,18],   
     'colsample_bytree': [0.3, 0.4, 0.5 ,0.6, 0.7],    
       
}

In [27]:
from sklearn.model_selection import GridSearchCV
xgb_grid=GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="f1_macro", n_jobs=-1, verbose = 2)
xgb_grid.fit(X_train, y_train)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.3, 0.4, 0.5, 0.6, 0.7],
                         'learning_rate': [0.1, 0.05],
                         'max_depth': [14, 15, 16, 17, 18],
                         'n_estimators': [400, 500, 600, 700, 800]},
             scoring='f1_macro', verbose=2)

In [28]:
print("best f1_macro : {0: .4f}".format(xgb_grid.best_score_))
print("best param : ",xgb_grid.best_params_)

best f1_macro :  0.5946
best param :  {'colsample_bytree': 0.4, 'learning_rate': 0.05, 'max_depth': 18, 'n_estimators': 800}


In [29]:
model = xgb_grid.best_estimator_

## 3. 정확도 및 f1 score

In [75]:
# 데이터프레임의 모든 변이 관련 열에 전처리 함수 적용
for col in X_test:
    X_test[col] = X_test[col].apply(lambda x: transform_mutation(x, col))

# 변환된 데이터프레임 출력
X_test.head()
val_str = X_test.astype(str).apply(' '.join, axis=1)

v_X_tfidf = tfidf_vectorizer.transform(val_str)

In [77]:
val_y = label_encoder.transform(y_test)

In [78]:
# Make predictions and evaluate
y_pred = model.predict(val_str)
accuracy = accuracy_score(val_y, y_pred)

print("Model Accuracy:", accuracy)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:0: object

In [73]:
from sklearn.metrics import f1_score

# 예측 수행
y_pred = model.predict(val_str)

# F1 score 계산
f1 = f1_score(y_test, y_pred, average='weighted')  # 또는 'macro', 'micro' 등 다른 옵션 선택 가능
print("F1 Score:", f1)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:SUBCLASS: object, ID: object, BRCA1: object, HMGB3: object, PLXNB2: object, NOTCH2: object, RYR2: object, ALB: object, BTG1: object, SMC1A: object, AQP7: object, PIM1: object, FBXW7: object, CDKN1A: object, ENPP2: object, AHNAK: object, COL6A3: object, PLEC: object, SOX9: object, CLTC: object, KIF3B: object, PKHD1: object, CREBBP: object, VHL: object, RNF213: object, AMOT: object, SPTB: object, TGFBR2: object, RYR1: object, SMC3: object, XDH: object, ALMS1: object, AKT1: object, MYH1: object, DLC1: object, SPOP: object, CELSR1: object, SPTAN1: object, LRP1: object, YWHAQ: object, MYH2: object, ATRX: object, COL5A1: object, PIK3CA: object, PCLO: object, SPTA1: object, NF1: object, FBN2: object, RELN: object, MXRA5: object, NOTCH1: object, KMT2D: object, NLRC5: object, TSC2: object, PE

# 추론

## data load

In [18]:

t_file_path = '../train_data/test.csv'
test = pd.read_csv(t_file_path)
test = test.fillna('WT')

## train 시킨 컬럼과 일치시키기

In [20]:
test = test[data_features.columns]

## transform_mutation 함수를 이용하여 전처리

In [21]:
# 데이터프레임의 모든 변이 관련 열에 전처리 함수 적용
for col in test:
    test[col] = test[col].apply(lambda x: transform_mutation(x, col))

# 변환된 데이터프레임 출력
test.head()


C:\Users\JINWOO\AppData\Local\Temp\ipykernel_25532\406577040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[col] = test[col].apply(lambda x: transform_mutation(x, col))


,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,AQP7,PIM1,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
0,,,,,,,,,,,...,,,,,,PABPC1,,,,
1,,,,,RYR2,,,,,,...,,,,,,,,,,HELZ2
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,


## vectorizer를 이용하여 encoding

In [22]:
test_str = test.astype(str).apply(' '.join, axis=1)

t_X_tfidf = tfidf_vectorizer.transform(test_str)

## 추론

In [32]:
predictions = model.predict(t_X_tfidf)

In [33]:
original_labels = label_encoder.inverse_transform(predictions)


# 대회 파일 저장

In [34]:
submisson = pd.read_csv("../train_data/sample_submission.csv")
submisson["SUBCLASS"] = original_labels
submisson.to_csv('../submission_data/baseline_input_colname_sampling_grid1.csv', encoding='UTF-8-sig', index=False)